In [ ]:
import numpy as np
import cv2
import json
from PIL import Image

import os
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def crop_save(img_path, ann_path, tensor_path=False):
    img_array = np.array(correct_exif_tags(Image.open(img_path)))
    with open(ann_path, 'r') as file:
        ann_json = json.load(file)
    buildings = []
    for house in ann_json['objects']:
        cl = int(house['classTitle'][1:2])
        if len(house['tags']) != 1 or cl == 4:
            continue
        floors = house['tags'][0]['value']
        [x1, y1], [x2, y2] = house['points']['exterior']
        build = img_array[y1:y2, x1:x2]
        buildings.append((build, floors, cl))
    if tensor_path:
        for i, trio in enumerate(buildings):
            name = tensor_path + f'__{i}h' + f'__{trio[2]}c' + f'__{trio[1]}f' + '.pth'
            matrix, changes = standardize_image(trio[0])
            matrix = torch.tensor(matrix, dtype=torch.uint8).permute(2,0,1)
            torch.save((matrix, trio[1], changes), name)       
    return buildings

def record(source_path, final_path):
    for uin in tqdm(os.listdir(source_path)):
        if '.' in uin:
            continue
        uin_path = source_path + '/' + uin
        ann_files = set([i[:-9] for i in os.listdir(uin_path + '/ann')])
        img_files = set([i[:-4] for i in os.listdir(uin_path + '/img')])
        files = list(ann_files & img_files)
        for file in tqdm(files, leave=False):
            img_path = uin_path + '/img/' + file + '.jpg'
            ann_path = uin_path + '/ann/' + file + '.jpg.json'
            tensor_path = final_path + ('/' if final_path != '' else '') + uin + '_' + file
            crop_save(img_path, ann_path, tensor_path)

In [1]:
source_path = ...
final_path = ...
record(source_path, final_path)

In [ ]:
class FilesDataset(Dataset):
    def __init__(self, file_paths, deterministic=False):
        self.file_paths = file_paths
        if deterministic:
            np.random.seed(1)
        np.random.shuffle(self.file_paths)
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        data = torch.load(file_path, weights_only=True)
        tensor_gray = (0.299 * data[0][0] + 0.587 * data[0][1] + 0.114 * data[0][2]).unsqueeze(0)
        return tensor_gray, data[1]

In [ ]:
all_files = [final_path + '/' + i for i in os.listdir(final_path)]
uins = list(set([i.split('/')[-1].split('_')[0] for i in all_files]))
train_uins, valid_uins = train_test_split(uins, test_size=0.2, random_state=42)
train_files = [i for i in all_files if i.split('/')[-1].split('_')[0] in train_uins]
valid_files = [i for i in all_files if i.split('/')[-1].split('_')[0] in valid_uins]

train_df = FilesDataset(train_files)
valid_df = FilesDataset(valid_files)

train_dataloader = DataLoader(train_df, batch_size=10, shuffle=True)
valid_dataloader = DataLoader(valid_df, batch_size=10, shuffle=False)

model = models.resnet50()
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(in_features=2048, out_features=1)

num_epochs = 100
device = 'cuda'
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-5, steps_per_epoch=len(train_dataloader), epochs=num_epochs, pct_start=0.1)

In [ ]:
model.to(device)
writer = SummaryWriter(log_dir="logs")

for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0.0
    for images, targets in tqdm(train_dataloader, leave=False):
        images, targets = images.to(device)/255, targets.to(device)/1
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()
    model.eval()
    valid_loss = 0.0
    for images, targets in tqdm(valid_dataloader, leave=False):
        images, targets = images.to(device)/255, targets.to(device)/1
        with torch.no_grad():
            outputs = model(images).squeeze(1)
            loss = criterion(outputs, targets)
            valid_loss += loss.item()
    writer.add_scalars("Loss", {"train": train_loss/len(train_dataloader), "val": valid_loss/len(valid_dataloader)}, epoch)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_dataloader):.4f}, Valid Loss: {valid_loss/len(valid_dataloader):.4f}")

writer.close()

In [ ]:
torch.save(model.state_dict(), ...)

In [ ]:
model.eval()

difs, y_true, y_pred = [0], [0], [0]
for image, target in tqdm(valid_df):
        image = image.unsqueeze(0).to(device)/255
        with torch.no_grad():
            out = round(model(image).item())
            if out < 1:
                out = 1
            dif = out - target
            difs.append(dif)
        y_true.append(target)
        y_pred.append(out)

In [ ]:
plt.hist(difs, bins = max(difs)-min(difs), edgecolor='black', color='grey')
plt.xlabel('Разница предсказания и реальности')
plt.ylabel('Частота')
plt.show()

In [ ]:
cm = confusion_matrix(y_true, y_pred)

# Настройки графика
fig, ax = plt.subplots(figsize=(20, 20))
cax = ax.matshow(cm, cmap=plt.cm.Blues)

# Добавляем цветовую шкалу
plt.colorbar(cax, shrink=0.7)

# Мелкие подписи осей
ax.set_xticks(np.arange(len(cm)))
ax.set_yticks(np.arange(len(cm)))
ax.set_xticklabels(np.arange(len(cm)), fontsize=8)
ax.set_yticklabels(np.arange(len(cm)), fontsize=8)

# Названия осей
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("Actual", fontsize=12)

plt.show()